## 1.2 Class Imbalance Handling

### ❓ Tại Sao Đây Là Vấn Đề Nghiêm Trọng?

**Phân tích class distribution:**
```
No Finding:    60,361 samples (53.84%) 😱
Infiltration:  19,894 samples (17.74%)
Atelectasis:   11,559 samples (10.31%)
...
Hernia:           227 samples (0.20%)  😱
```

**Hậu quả:**
- Model bias về "No Finding" → Predict "No Finding" cho mọi case
- Rare diseases (Hernia, Pneumonia) bị ignore → Nguy hiểm trong medical application!
- AUC tổng thể có thể cao nhưng per-class performance kém

### 💡 Giải Pháp: Multi-Strategy Approach

#### Strategy 1: Focal Loss
**Tại sao:** Tự động focus vào hard/rare examples

$$FL(p_t) = -\alpha_t(1-p_t)^\gamma \log(p_t)$$

- $\gamma = 2$: Down-weight easy examples
- $\alpha = 0.25$: Balance positive/negative

#### Strategy 2: Class Weights
**Tại sao:** Penalty cao hơn khi predict sai rare classes

$$w_i = \frac{N_{total} - N_positive_i}{N_positive_i}$$

#### Strategy 3: Weighted Sampling
**Tại sao:** Đảm bảo mỗi batch có representation của rare classes

### 📈 Expected Impact
- **+3-5% AUC** on rare classes (Hernia, Pneumonia, Fibrosis)
- More balanced predictions across all diseases
- Clinically safer model

In [ ]:
class FocalLoss(nn.Module):
    """
    Focal Loss for multi-label classification
    
    Paper: "Focal Loss for Dense Object Detection" (Lin et al., 2017)
    Adapted for multi-label medical imaging
    
    Args:
        alpha (float): Weighting factor [0, 1]
        gamma (float): Focusing parameter >= 0
        pos_weight (Tensor): Positive class weights for each class
    
    Intuition:
    - gamma=0: Standard BCE loss
    - gamma↑: More focus on hard examples
    - Easy examples (pt → 1) get down-weighted by (1-pt)^gamma
    """
    def __init__(self, alpha=0.25, gamma=2.0, pos_weight=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.pos_weight = pos_weight
    
    def forward(self, inputs, targets):
        """
        Args:
            inputs: (N, C) logits (before sigmoid)
            targets: (N, C) binary labels
        """
        # BCE loss with logits
        BCE_loss = F.binary_cross_entropy_with_logits(
            inputs, targets, 
            pos_weight=self.pos_weight,
            reduction='none'
        )
        
        # Probability of correct class
        pt = torch.exp(-BCE_loss)
        
        # Focal term: (1-pt)^gamma
        # Khi pt → 1 (easy): focal_term → 0 (down-weight)
        # Khi pt → 0 (hard): focal_term → 1 (keep weight)
        focal_term = (1 - pt) ** self.gamma
        
        # Final loss
        focal_loss = self.alpha * focal_term * BCE_loss
        
        return focal_loss.mean()


class WeightedBCELoss(nn.Module):
    """
    BCE Loss với class weights
    
    Tính pos_weight dựa trên class frequency:
    pos_weight[i] = (N_total - N_positive[i]) / N_positive[i]
    
    Classes hiếm → pos_weight cao → penalty cao khi predict sai
    """
    def __init__(self, pos_weight=None):
        super(WeightedBCELoss, self).__init__()
        self.pos_weight = pos_weight
    
    def forward(self, inputs, targets):
        return F.binary_cross_entropy_with_logits(
            inputs, targets,
            pos_weight=self.pos_weight
        )


class LabelSmoothingBCE(nn.Module):
    """
    Label Smoothing for BCE Loss
    
    Regularization technique:
    - Original: target ∈ {0, 1}
    - Smoothed: target ∈ [ε, 1-ε]
    
    Benefits:
    - Prevent overconfident predictions
    - Handle label noise (~10% in NIH dataset)
    - Better calibration
    """
    def __init__(self, smoothing=0.1, pos_weight=None):
        super(LabelSmoothingBCE, self).__init__()
        self.smoothing = smoothing
        self.pos_weight = pos_weight
    
    def forward(self, inputs, targets):
        # Smooth labels: 1 → 1-ε, 0 → ε
        targets_smooth = targets * (1 - self.smoothing) + 0.5 * self.smoothing
        
        return F.binary_cross_entropy_with_logits(
            inputs, targets_smooth,
            pos_weight=self.pos_weight
        )


def compute_class_weights(df, disease_columns):
    """
    Tính class weights dựa trên frequency
    
    Formula: w_i = (N_total - N_positive_i) / N_positive_i
    
    Example:
    - Hernia: 227 samples → weight = (100000 - 227) / 227 ≈ 440
    - No Finding: 60361 samples → weight = (100000 - 60361) / 60361 ≈ 0.66
    """
    class_counts = df[disease_columns].sum().values
    total_samples = len(df)
    
    # Inverse frequency weighting
    pos_weights = (total_samples - class_counts) / np.maximum(class_counts, 1)
    
    # Normalize to prevent extreme values
    pos_weights = np.clip(pos_weights, 0.5, 100)  # Clip to reasonable range
    
    return torch.FloatTensor(pos_weights)


def compute_sample_weights(df, disease_columns):
    """
    Tính sampling weights cho WeightedRandomSampler
    
    Strategy: Sample có rare disease → weight cao → probability sampling cao
    
    Returns:
        weights: (N,) array of sampling weights
    """
    # Inverse class frequency
    class_counts = df[disease_columns].sum().values
    class_weights = 1.0 / np.maximum(class_counts, 1)
    
    # Sample weight = max class weight của các diseases có trong sample
    sample_weights = []
    for _, row in df.iterrows():
        labels = row[disease_columns].values
        # Weight = sum of weights for positive classes
        weight = np.sum(class_weights * labels)
        if weight == 0:  # No positive labels
            weight = class_weights.min()
        sample_weights.append(weight)
    
    return np.array(sample_weights)


print("✅ Loss functions implemented:")
print("   1. FocalLoss (α=0.25, γ=2.0)")
print("   2. WeightedBCELoss")
print("   3. LabelSmoothingBCE (ε=0.1)")
print("\n📊 Class weighting strategies:")
print("   - compute_class_weights(): For loss functions")
print("   - compute_sample_weights(): For WeightedRandomSampler")